# sleep log creation

In [51]:
import pandas as pd
from datetime import datetime, timedelta
import os

In [52]:
obs_sleep = "/mnt/lss/Projects/BOOST/ObservationalStudy/3-experiment/data/act-obs-test/sleep_log_observational.csv"
obs = pd.read_csv(obs_sleep)
obs

,ID,D1_date,D1_inbed,D1_wakeup,D2_date,D2_inbed,D2_wakeup,D3_date,D3_inbed,D3_wakeup,...,D7_wakeup,D8_date,D8_inbed,D8_wakeup,D9_date,D9_inbed,D9_wakeup,D10_date,D10_inbed,D10_wakeup
0,sub-7062_ses-1_accel,2025-02-22,22:15:00,05:55:00,2025-02-23,22:15:00,04:50:00,2025-02-24,21:45:00,04:30:00,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,sub-7146_ses-1_accel,2025-05-05,09:00:00,14:30:00,2025-05-06,09:00:00,15:00:00,2025-05-07,09:00:00,15:00:00,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,sub-7001_ses-1_accel,2024-09-06,00:00:00,06:00:00,2024-09-07,21:00:00,08:30:00,2024-09-08,22:00:00,06:45:00,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,sub-7002_ses-1_accel,2024-08-08,22:46:00,06:55:00,2024-08-09,22:46:00,06:59:00,2024-08-10,22:39:00,07:53:00,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,sub-7003_ses-1_accel,2024-08-21,23:20:00,06:20:00,2024-08-22,21:30:00,07:01:00,2024-08-23,11:30:00,09:02:00,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
121,sub-7159_ses-1_accel,2025-06-05,22:40:00,07:10:00,2025-06-06,22:15:00,09:00:00,2025-06-07,23:30:00,09:45:00,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
122,sub-7160_ses-1_accel,2025-06-04,22:34:00,06:00:00,2025-06-05,21:43:00,05:56:00,2025-06-06,22:34:00,06:34:00,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
123,sub-7165_ses-1_accel,2025-06-12,23:30:00,05:30:00,2025-06-13,23:00:00,05:45:00,2025-06-14,23:00:00,05:45:00,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
124,sub-7161_ses-1_accel,2025-06-10,22:10:00,06:00:00,2025-06-11,22:00:00,06:00:00,2025-06-12,21:52:00,06:30:00,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [53]:
obs_subs = ["sub-7057_ses-1_accel", 'sub-7058_ses-1_accel', 'sub-7059_ses-1_accel']
obs['ID'] = obs['ID'].astype(str)
obs = obs[obs['ID'].isin(obs_subs)]

In [54]:
sub3 = ["22:00:00", "05:00:00", "21:00:00", "04:30:00", "21:30:00"]
sub2 = ["22:45:00", "07:30:00", "22:30:00", "08:15:00", "23:00:00"]
sub1 = ["22:00:00", "07:15:00", "22:15:00", "07:00:00", "21:55:00"]

In [55]:
from datetime import datetime, timedelta
import pandas as pd

subjects = {"sub1": sub1, "sub2": sub2, "sub3": sub3}

# Build new rows
new_rows = []
base_date = datetime(2025, 8, 18)

for idx, (name, times) in enumerate(subjects.items(), start=1):
    row = {}
    row["ID"] = f"sub-{idx}_ses-1_accel"
    
    # handle full inbed/wakeup pairs
    for day in range(len(times) // 2):
        dcol = f"D{day+1}_date"
        icol = f"D{day+1}_inbed"
        wcol = f"D{day+1}_wakeup"
        row[dcol] = (base_date + timedelta(days=day)).date().isoformat()
        row[icol] = times[2 * day]
        row[wcol] = times[2 * day + 1]
    
    # handle leftover "inbed" if odd length
    if len(times) % 2 == 1:
        leftover_day = len(times) // 2 + 1
        dcol = f"D{leftover_day}_date"
        icol = f"D{leftover_day}_inbed"
        row[dcol] = (base_date + timedelta(days=leftover_day - 1)).date().isoformat()
        row[icol] = times[-1]
        # no wakeup available
    
    new_rows.append(row)

# Append to DataFrame, aligning by column names
obs = pd.concat([obs, pd.DataFrame(new_rows)], ignore_index=True)

print(obs.tail())


                     ID     D1_date  D1_inbed D1_wakeup     D2_date  D2_inbed  \
1  sub-7058_ses-1_accel  2025-02-21  22:30:00  07:20:00  2025-02-22  21:30:00   
2  sub-7059_ses-1_accel  2025-02-25  22:30:00  06:20:00  2025-02-26  22:40:00   
3     sub-1_ses-1_accel  2025-08-18  22:00:00  07:15:00  2025-08-19  22:15:00   
4     sub-2_ses-1_accel  2025-08-18  22:45:00  07:30:00  2025-08-19  22:30:00   
5     sub-3_ses-1_accel  2025-08-18  22:00:00  05:00:00  2025-08-19  21:00:00   

  D2_wakeup     D3_date  D3_inbed D3_wakeup  ... D7_wakeup D8_date D8_inbed  \
1  07:00:00  2025-02-23  21:15:00  06:00:00  ...       NaN     NaN      NaN   
2  07:30:00  2025-02-27  22:30:00  05:00:00  ...       NaN     NaN      NaN   
3  07:00:00  2025-08-20  21:55:00       NaN  ...       NaN     NaN      NaN   
4  08:15:00  2025-08-20  23:00:00       NaN  ...       NaN     NaN      NaN   
5  04:30:00  2025-08-20  21:30:00       NaN  ...       NaN     NaN      NaN   

  D8_wakeup D9_date D9_inbed D9_wakeup

In [57]:
# figure out which columns are actually used (non-empty in any row)
used_cols = [c for c in obs.columns if not obs[c].isna().all()]

# restrict the DataFrame to those columns
obs = obs[used_cols]

print(obs.head())

                     ID     D1_date  D1_inbed D1_wakeup     D2_date  D2_inbed  \
0  sub-7057_ses-1_accel  2025-02-15  22:40:00  08:30:00  2025-02-17  22:00:00   
1  sub-7058_ses-1_accel  2025-02-21  22:30:00  07:20:00  2025-02-22  21:30:00   
2  sub-7059_ses-1_accel  2025-02-25  22:30:00  06:20:00  2025-02-26  22:40:00   
3     sub-1_ses-1_accel  2025-08-18  22:00:00  07:15:00  2025-08-19  22:15:00   
4     sub-2_ses-1_accel  2025-08-18  22:45:00  07:30:00  2025-08-19  22:30:00   

  D2_wakeup     D3_date  D3_inbed D3_wakeup     D4_date  D4_inbed D4_wakeup  \
0  09:30:00  2025-02-19  01:00:00  11:30:00  2025-02-20  00:00:00  10:30:00   
1  07:00:00  2025-02-23  21:15:00  06:00:00  2025-02-24  21:30:00  06:00:00   
2  07:30:00  2025-02-27  22:30:00  05:00:00  2025-03-01  01:20:00  09:20:00   
3  07:00:00  2025-08-20  21:55:00       NaN         NaN       NaN       NaN   
4  08:15:00  2025-08-20  23:00:00       NaN         NaN       NaN       NaN   

      D5_date  D5_inbed D5_wakeup     

In [59]:
obs.to_csv("/mnt/lss/Users/zak/ggir/sleep.csv", index=False)